# Introduction
This python notebook contains the solutions to the exercices found in the following: https://adventofcode.com/2015

# --- Day 1: Not Quite Lisp ---
Santa is trying to deliver presents in a large apartment building, but he can't find the right floor - the directions he got are a little confusing. He starts on the ground floor (floor 0) and then follows the instructions one character at a time.

An opening parenthesis, (, means he should go up one floor, and a closing parenthesis, ), means he should go down one floor.

The apartment building is very tall, and the basement is very deep; he will never find the top or bottom floors.

For example:

- (()) and ()() both result in floor 0.
- ((( and (()(()( both result in floor 3.
- ))((((( also results in floor 3.
- ()) and ))( both result in floor -1 (the first basement level).
- ))) and )())()) both result in floor -3.

To what floor do the instructions take Santa?

In [7]:
test = ')())())'

In [3]:
def translateDirections(confusing_str):
    position = 0
    for char in confusing_str:
        if char == '(':
            position += 1
        if char == ')':
            position -= 1
    
    return position

In [18]:
print(f'Test should be equal to -3, and it is {translateDirections(test)}')
confusing_direc = open('Day1_ConfusingDirections.txt','r')
confusing_str = confusing_direc.readlines()[0]
print(f'Santa needs to got to floor number {translateDirections(confusing_str)}')

Test should be equal to -3, and it is -3
Santa needs to got to floor number 232


## --- Part Two ---

Now, given the same instructions, find the position of the first character that causes him to enter the basement (floor -1). The first character in the instructions has position 1, the second character has position 2, and so on.

For example:
- ) causes him to enter the basement at character position 1.
- ()()) causes him to enter the basement at character position 5.

What is the position of the character that causes Santa to first enter the basement?

In [21]:
def whenInBasement(confusing_str):
    position = 0
    when = 0
    for char in confusing_str:
        if char == '(':
            position += 1
        if char == ')':
            position -= 1
        when += 1
        if position < 0:
            break
        
    return when

In [24]:
print(f'Santa entered the basement at position {whenInBasement(confusing_str)}')

Santa entered the basement at position 1783


# --- Day 2: I Was Told There Would Be No Math ---

The elves are running low on wrapping paper, and so they need to submit an order for more. They have a list of the dimensions (length l, width w, and height h) of each present, and only want to order exactly as much as they need.

Fortunately, every present is a box (a perfect right rectangular prism), which makes calculating the required wrapping paper for each gift a little easier: find the surface area of the box, which is 2*l*w + 2*w*h + 2*h*l. The elves also need a little extra paper for each present: the area of the smallest side.

For example:
- A present with dimensions 2x3x4 requires 2x6 + 2x12 + 2x8 = 52 square feet of wrapping paper plus 6 square feet of slack, for a total of 58 square feet.
- A present with dimensions 1x1x10 requires 2x1 + 2x10 + 2x10 = 42 square feet of wrapping paper plus 1 square foot of slack, for a total of 43 square feet.
All numbers in the elves' list are in feet. How many total square feet of wrapping paper should they order?

In [111]:
def wrappingPaper(l, w, h):
    present_area = 2 * (l*w + w*h + h*l) + min(l*w, w*h, h*l)
    return present_area

def howMuchPaper(present_list):
    total_paper = 0
    for present in present_list:
        present_dimensions = tuple([int(dimension) for dimension in present.split('x')])
        total_paper += wrappingPaper(*present_dimensions) 
    return total_paper

In [112]:
present_list = open('Day2_PresentDimensions.txt', 'r')
present_list = present_list.readlines()
howMuchPaper(present_list)

1606483

## --- Part Two ---

The elves are also running low on ribbon. Ribbon is all the same width, so they only have to worry about the length they need to order, which they would again like to be exact.

The ribbon required to wrap a present is the shortest distance around its sides, or the smallest perimeter of any one face. Each present also requires a bow made out of ribbon as well; the feet of ribbon required for the perfect bow is equal to the cubic feet of volume of the present. Don't ask how they tie the bow, though; they'll never tell.

For example:
- A present with dimensions 2x3x4 requires 2+2+3+3 = 10 feet of ribbon to wrap the present plus 2x3x4 = 24 feet of ribbon for the bow, for a total of 34 feet.
- A present with dimensions 1x1x10 requires 1+1+1+1 = 4 feet of ribbon to wrap the present plus 1x1x10 = 10 feet of ribbon for the bow, for a total of 14 feet.

How many total feet of ribbon should they order?

In [148]:
def ribbonLength(l,w,h):
    ribbon_around = min (2*(l+w),2*(w+h),2*(l+h))
    ribbon_bow = l*w*h
    return ribbon_around + ribbon_bow

def howMuchRibbon(present_list):
    total_ribbon = 0
    for present in present_list:
        present_dimensions = tuple([int(dimension) for dimension in present.split('x')])
        total_ribbon += ribbonLength(*present_dimensions) 
    return total_ribbon    

In [149]:
present_list = open('Day2_PresentDimensions.txt', 'r')
present_list = present_list.readlines()
howMuchRibbon(present_list)

3842356

# --- Day 3: Perfectly Spherical Houses in a Vacuum ---

Santa is delivering presents to an infinite two-dimensional grid of houses.

He begins by delivering a present to the house at his starting location, and then an elf at the North Pole calls him via radio and tells him where to move next. Moves are always exactly one house to the north (^), south (v), east (>), or west (<). After each move, he delivers another present to the house at his new location.

However, the elf back at the north pole has had a little too much eggnog, and so his directions are a little off, and Santa ends up visiting some houses more than once. How many houses receive at least one present?

For example:

- '> delivers presents to 2 houses: one at the starting location, and one to the east.'
- ^>v< delivers presents to 4 houses in a square, including twice to the house at his starting/ending location.
- ^v^v^v^v^v delivers a bunch of presents to some very lucky children at only 2 houses.

In [206]:
def uniqueHouses(route):
    direc = {'<' : -1,'>' : 1, '^' : 1, 'v' : -1}
    start = [0,0]
    house_log = [(0,0)]

    # Follow each_step and record all house coordinates
    for each_step in route:
        if each_step in '<>': # Horizontal move
            start[0] += direc[each_step]
        if each_step in '^v': # Vertical move
            start[1] += direc[each_step]
        # Log current house position
        house_log += [tuple(start)]
    
    # Use a set to filter unique house positions and get the count
    unique_houses = len(set(house_log))
    
    return unique_houses

In [205]:
f = open('Day3_SantaRoute', 'r')
route = f.readlines()[0]
print(f'Santa delivered to {uniqueHouses(route)} unique houses')

Santa delivered to 2081 unique houses


## --- Part Two ---

The next year, to speed up the process, Santa creates a robot version of himself, Robo-Santa, to deliver presents with him.

Santa and Robo-Santa start at the same location (delivering two presents to the same starting house), then take turns moving based on instructions from the elf, who is eggnoggedly reading from the same script as the previous year.

This year, how many houses receive at least one present?

For example:

- ^v delivers presents to 3 houses, because Santa goes north, and then Robo-Santa goes south.
- ^>v< now delivers presents to 3 houses, and Santa and Robo-Santa end up back where they started.
- ^v^v^v^v^v now delivers presents to 11 houses, with Santa going one direction and Robo-Santa going the other.

In [215]:
def uniqueHouses_SnR(route):
    direc = {'<' : -1,'>' : 1, '^' : 1, 'v' : -1}

    # Santa and Robo-Santa starting position
    start_santa = [0,0]
    start_robo = [0,0]

    # House position log
    house_log = [(0,0)]

    #Follow each_step and record all house coordinates
    santa_turn = True
    
    for each_step in route:
        #Santa turn to move
        if santa_turn:
            if each_step in '<>':
                start_santa[0] += direc[each_step]
            if each_step in '^v':
                start_santa[1] += direc[each_step]
                
            house_log += [tuple(start_santa)]
            santa_turn = False

        # Robo turn to move
        else:
            if each_step in '<>':
                start_robo[0] += direc[each_step]
            if each_step in '^v':
                start_robo[1] += direc[each_step]
            
            house_log += [tuple(start_robo)]
            santa_turn = True

    # Use a set to filter unique house positions and get the count
    unique_houses = len(set(house_log))
    
    return unique_houses

In [219]:
print(f'Santa and Robo-Santa delivered to {uniqueHouses_SnR(route)} unique houses')

Santa and Robo-Santa delivered to 2341 unique houses


# --- Day 4: The Ideal Stocking Stuffer ---

Santa needs help mining some AdventCoins (very similar to bitcoins) to use as gifts for all the economically forward-thinking little girls and boys.

To do this, he needs to find MD5 hashes which, in hexadecimal, start with at least five zeroes. The input to the MD5 hash is some secret key (your puzzle input, given below) followed by a number in decimal. To mine AdventCoins, you must find Santa the lowest positive number (no leading zeroes: 1, 2, 3, ...) that produces such a hash.

For example:
- If your secret key is abcdef, the answer is 609043, because the MD5 hash of abcdef609043 starts with five zeroes (000001dbbfa...), and it is the lowest such number to do so.
- If your secret key is pqrstuv, the lowest number it combines with to make an MD5 hash starting with five zeroes is 1048970; that is, the MD5 hash of pqrstuv1048970 looks like 000006136ef....


In [42]:
import hashlib

def lowestNum(secret_key):
    n = 0

    #Loop will run until the hexadecimal hashed str starts with 00000
    while True:  
        #Generate hash
        hashed = hashlib.md5(f'{secret_key}{n}'.encode())

        #Check if 00000 is in the hexadecimal form
        if hashed.hexdigest().startswith('00000'):
            return n
        
        n +=1

In [45]:
lowestNum('bgvyzdsv')

254575

## --- Part Two ---

Now find one that starts with six zeroes.

In [46]:
def lowestNum(secret_key):
    n = 0

    #Loop will run until the hexadecimal hashed str starts with 00000
    while True:  
        #Generate hash
        hashed = hashlib.md5(f'{secret_key}{n}'.encode())

        #Check if 00000 is in the hexadecimal form
        if hashed.hexdigest().startswith('000000'):
            return n
        
        n +=1

lowestNum('bgvyzdsv')

1038736

# --- Day 5: Doesn't He Have Intern-Elves For This? ---

Santa needs help figuring out which strings in his text file are naughty or nice.

A nice string is one with all of the following properties:

It contains at least three vowels (aeiou only), like aei, xazegov, or aeiouaeiouaeiou.
It contains at least one letter that appears twice in a row, like xx, abcdde (dd), or aabbccdd (aa, bb, cc, or dd).
It does not contain the strings ab, cd, pq, or xy, even if they are part of one of the other requirements.
For example:
- ugknbfddgicrmopn is nice because it has at least three vowels (u...i...o...), a double letter (...dd...), and none of the disallowed substrings.
- aaa is nice because it has at least three vowels and a double letter, even though the letters used by different rules overlap.
- jchzalrnumimnmhp is naughty because it has no double letter.
- haegwjzuvuyypxyu is naughty because it contains the string xy.
- dvszwmarrgswjxmb is naughty because it contains only one vowel.

How many strings are nice?

In [162]:
import re

def isNice(string):

    # Check for prohibited letter convinations
    for p_letter in ['ab','cd', 'pq', 'xy']:
        if p_letter in string:
            return False
    
    # Check for presence of at least 3 vowels
    all_vowes = re.findall('[a,e,i,o,u]', string)
    if len(all_vowes) < 3:
        return False

    # Check for precense of double letter
    prev_letter = None
    for letter in string:
        if letter == prev_letter:
            return True # Passed all checks, so nice string
        prev_letter = letter
    
    return False #Did not pass all checks
    

In [216]:
f = open('Day5_NiceNaughtyStr', 'r')
str_list = f.readlines()

In [217]:
print(f'In Santas list there are {len(list(filter(isNice, str_list)))} nice strings')

In Santas list there are 258 nice strings


## --- Part Two ---

Realizing the error of his ways, Santa has switched to a better model of determining whether a string is naughty or nice. None of the old rules apply, as they are all clearly ridiculous.

Now, a nice string is one with all of the following properties:

It contains a pair of any two letters that appears at least twice in the string without overlapping, like xyxy (xy) or aabcdefgaa (aa), but not like aaa (aa, but it overlaps).
It contains at least one letter which repeats with exactly one letter between them, like xyx, abcdefeghi (efe), or even aaa.
For example:
- qjhvhtzxzqqjkmpb is nice because is has a pair that appears twice (qj) and a letter that repeats with exactly one letter between them (zxz).
- xxyxx is nice because it has a pair that appears twice and a letter that repeats with one between, even though the letters used by each rule overlap.
- uurcxstgmygtbstg is naughty because it has a pair (tg) but no repeat with a single letter between them.
- ieodomkazucvgmuy is naughty because it has a repeating letter with one between (odo), but no pair that appears twice.

How many strings are nice under these new rules?

In [211]:
def isNice_v2(string):
    double_pair = False
    sandwich = False
    
    # Check for a double of the same pair
    if any(string[i:i+2] in string[i+2:] for i in range(len(string)-3)):
        double_pair = True

    # Check for same letter doing sandwich 
    if any(string[i] == string[i+2] for i in range(len(string)-2)):
        sandwich = True

    # If both condition true, then str is nice
    if double_pair and sandwich:
        return True
    
    return False 

In [221]:
print(f'Under the new rules there are {len(list(filter(isNice_v2, str_list)))} nice strings')

Under the new rules there are 53 nice strings


# --- Day 6: Probably a Fire Hazard ---

Because your neighbors keep defeating you in the holiday house decorating contest year after year, you've decided to deploy one million lights in a 1000x1000 grid.

Furthermore, because you've been especially nice this year, Santa has mailed you instructions on how to display the ideal lighting configuration.

Lights in your grid are numbered from 0 to 999 in each direction; the lights at each corner are at 0,0, 0,999, 999,999, and 999,0. The instructions include whether to turn on, turn off, or toggle various inclusive ranges given as coordinate pairs. Each coordinate pair represents opposite corners of a rectangle, inclusive; a coordinate pair like 0,0 through 2,2 therefore refers to 9 lights in a 3x3 square. The lights all start turned off.

To defeat your neighbors this year, all you have to do is set up your lights by doing the instructions Santa sent you in order.

For example:
- turn on 0,0 through 999,999 would turn on (or leave on) every light.
- toggle 0,0 through 999,0 would toggle the first line of 1000 lights, turning off the ones that were on, and turning on the ones that were off.
- turn off 499,499 through 500,500 would turn off (or leave off) the middle four lights.

After following the instructions, how many lights are lit?

In [513]:
import numpy as np

def lightingConfig(command, sqr_corners, grid):
    'Updates the grid based on the command and the given area'
    
    top_left, bottom_right = sqr_corners
    # Select the area of the grid to modify
    selected_area = np.flip(grid, 0)[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
    
    if command == 'on':
        selected_area[:] = 1
    elif command == 'off':
        selected_area[:] = 0
    elif command == 'toggle':
        selected_area[:] = 1 - selected_area
        
    return grid

def readCommand(string):
    'Parses a single command string into a command and its coordinate ranges'
    
    #Remove 'turn' from command 'turn on' and 'turn off'
    if len(string.split()) == 5:
        string = string.split()[1:]
    else:
        string= string.split()
    
    string.remove('through')

    #Select and format parts of the string
    command = string[0]
    
    point1 = tuple(int(str) for str in string[1].split(','))
    point2 = tuple(int(str)+1 for str in string[2].split(','))

    return command, [point1, point2]
            

def howManyOnLights(string_list):
    'Computes the total number of lights that are on after applying all commands'
    
    grid = np.zeros((1000, 1000), dtype=int)
    
    for string in string_list:
        lightingConfig(*readCommand(string),grid)

    return np.sum(grid)

In [512]:
f = open('Day6_LightingPlan', 'r')
f = f.readlines()
howManyOnLights(f)

377891

## --- Part Two ---

You just finish implementing your winning light pattern when you realize you mistranslated Santa's message from Ancient Nordic Elvish.

The light grid you bought actually has individual brightness controls; each light can have a brightness of zero or more. The lights all start at zero.

The phrase turn on actually means that you should increase the brightness of those lights by 1.

The phrase turn off actually means that you should decrease the brightness of those lights by 1, to a minimum of zero.

The phrase toggle actually means that you should increase the brightness of those lights by 2.

What is the total brightness of all lights combined after following Santa's instructions?

For example:
- turn on 0,0 through 0,0 would increase the total brightness by 1.
- toggle 0,0 through 999,999 would increase the total brightness by 2000000.


In [519]:
def lightingConfig_v2(command, sqr_corners, grid):
    'Updates the grid based on the command and the given area'
    
    top_left, bottom_right = sqr_corners
    # Select the area of the grid to modify
    selected_area = np.flip(grid, 0)[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
    
    if command == 'on':
        selected_area[:] += 1
    elif command == 'off':
        selected_area[~np.isin(selected_area, 0)] -= 1
    elif command == 'toggle':
        selected_area[:] += 2


def howManyOnLights_v2(string_list):
    'Computes the total number of lights that are on after applying all commands'
    
    grid = np.zeros((1000, 1000), dtype=int)
    
    for string in string_list:
        lightingConfig_v2(*readCommand(string),grid)

    return np.sum(grid)

In [520]:
howManyOnLights_v2(f)

14110788

# --- Day 7: Some Assembly Required ---
